In [1]:
import unetgee as ug
from update_training_record import *
import pandas as pd
import os
import re

2024-08-14 09:59:17.239528: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 09:59:17.399602: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 09:59:21.743780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 09:59:21.785754: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bu

In [16]:
# Step 1: Load the CSV file and extract the "Model Name" column
def load_model_names(csv_path):
    df = pd.read_csv(csv_path)
    return set(df['Model Name'].astype(str))  # Convert to string for matching purposes

# Step 2: List all folders in the directory
def list_folders(base_path):
    return [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

# Step 3: Check if the folder's model name is in the CSV
def check_folders(csv_model_names, folders, return_existing=False):
    pattern = re.compile(r"10bands_LAEA_samples_(\d+)")
    print(csv_model_names)
    print(folders)
    matched_folders = {}
    for folder in folders:
        match = pattern.search(folder)
        if return_existing:
            if match and match.group(1) in csv_model_names:
                matched_folders[match.group(1)] = folder
        else :
            if match and match.group(1) not in csv_model_names:
                matched_folders[match.group(1)] = folder
    return matched_folders

In [18]:
csv_path = '/WORK/Codes/global_lake_area/training_records.csv'
parent_folder_path = '/WORK/SSD_Data/global_lake_area/unet_sample_10bands_laea_per_basin'

csv_model_names = load_model_names(csv_path)
folders = list_folders(parent_folder_path)

matched_folders = check_folders(csv_model_names, folders, return_existing=True)
print(matched_folders)

{'4020050220', '3020008670', '9020000010', '6020021870', '1020034170', '1020035180', '6020029280', '7020024600', '4020024190', '4020015090', '1020011530', '2020041390', '3020009320', '7020038340', '1020000010', '4020034510', '8020000010', '2020033490', '4020050210', '4020000010', '4020050290', '4020050470', '7020065090', '7020046750', '2020057170', '7020014250', '8020008900', '7020000010', '5020015660', '2020065840', '8020032840', '8020044560', '2020071190', '6020000010', '5020037270', '4020006940', '2020000010', '7020021430', '1020021940', '2020024230', '8020010700', '5020000010', '6020017370', '1020027430', '6020008320', '3020003790', '6020014330', '3020005240', '8020020760', '3020024310', '2020018240', '6020006540', '2020003440', '3020000010', '7020047840', '5020049720', '1020018110', '5020082270', '1020040190', '8020022890'}
['10bands_LAEA_samples_1020000010', '10bands_LAEA_samples_1020011530', '10bands_LAEA_samples_1020018110', '10bands_LAEA_samples_1020021940', '10bands_LAEA_samp

In [4]:

for hybas_id, folder in matched_folders.items():
    hybas_id = int(hybas_id)
    current_hybas_id = folder.split('_')[-1]
    training_size, eval_size = ug.read_samples(
        training_pattern='training',
        eval_pattern='eval',
        local_sample_folder=os.path.join(parent_folder_path, folder)+'/',
        input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'GSW_Occurrence', 'GSW_Recurrence'],
        input_band_scaling_factors=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.01, 0.01],
        response=['Water_GSW'],
        kernel_size=128,
        batch_size=128,
        only_sizes=True,
        compression_type='GZIP'
    )
    print('Training size:', training_size)
    print('Eval size:', eval_size)

    update_training_record(hybas_id, trained_status='N', training_set_size=training_size, evaluation_set_size=eval_size)

2024-04-19 09:31:19.937136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 09:31:19.937514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 09:31:19.937761: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 09:31:20.490249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-19 09:31:20.490721: I tensorflow/compile

training size: 140335, eval_size: 34946
training size: 140335, eval_size: 34946
Training size: 140335
Eval size: 34946
training size: 100964, eval_size: 25903
training size: 100964, eval_size: 25903
Training size: 100964
Eval size: 25903
training size: 123338, eval_size: 30881
training size: 123338, eval_size: 30881
Training size: 123338
Eval size: 30881
training size: 12947, eval_size: 3337
training size: 12947, eval_size: 3337
Training size: 12947
Eval size: 3337
training size: 88253, eval_size: 21994
training size: 88253, eval_size: 21994
Training size: 88253
Eval size: 21994
training size: 16572, eval_size: 4747
training size: 16572, eval_size: 4747
Training size: 16572
Eval size: 4747
training size: 55058, eval_size: 13328
training size: 55058, eval_size: 13328
Training size: 55058
Eval size: 13328
training size: 108574, eval_size: 28365
training size: 108574, eval_size: 28365
Training size: 108574
Eval size: 28365
training size: 85089, eval_size: 21373
training size: 85089, eval_

In [19]:

for hybas_id, folder in matched_folders.items():
    hybas_id = int(hybas_id)
    current_hybas_id = folder.split('_')[-1]
    test_size, _size = ug.read_samples(
        training_pattern='test',
        eval_pattern='test',
        local_sample_folder=os.path.join(parent_folder_path, folder)+'/',
        input_bands=['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'GSW_Occurrence', 'GSW_Recurrence'],
        input_band_scaling_factors=[0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.01, 0.01],
        response=['Water_GSW'],
        kernel_size=128,
        batch_size=128,
        only_sizes=True,
        compression_type='GZIP'
    )
    print('Test size:', test_size)

    update_training_record(hybas_id, test_set_size=test_size)

2024-08-14 10:06:06.748887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 10:06:06.749421: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 10:06:06.749723: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 10:06:07.947161: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 10:06:07.947747: I tensorflow/compile

training size: 33089, eval_size: 33089
training size: 33089, eval_size: 33089
Test size: 33089
training size: 29666, eval_size: 29666
training size: 29666, eval_size: 29666
Test size: 29666
training size: 24084, eval_size: 24084
training size: 24084, eval_size: 24084
Test size: 24084
training size: 19432, eval_size: 19432
training size: 19432, eval_size: 19432
Test size: 19432
training size: 30316, eval_size: 30316
training size: 30316, eval_size: 30316
Test size: 30316
training size: 28484, eval_size: 28484
training size: 28484, eval_size: 28484
Test size: 28484
training size: 24671, eval_size: 24671
training size: 24671, eval_size: 24671
Test size: 24671
training size: 33506, eval_size: 33506
training size: 33506, eval_size: 33506
Test size: 33506
training size: 15836, eval_size: 15836
training size: 15836, eval_size: 15836
Test size: 15836
training size: 19530, eval_size: 19530
training size: 19530, eval_size: 19530
Test size: 19530
training size: 9771, eval_size: 9771
training size